## Data Source
- CaseDeathTest: $\href{https://data.chhs.ca.gov/dataset/covid-19-time-series-metrics-by-county-and-state}{\text{Statewide COVID-19 Cases Deaths Tests}}$

Download date: 9/6/2022

In [1]:
import numpy as np
import pandas as pd

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
CaseDeathTest = pd.read_csv("covid_cases_tests_2022-09-06.csv")

In [4]:
# check original length
print("Length of CaseDeathTest", len(CaseDeathTest))

Length of CaseDeathTest 57889


In [5]:
# check head
CaseDeathTest.head()

,date,area,area_type,population,cases,cumulative_cases,deaths,cumulative_deaths,total_tests,cumulative_total_tests,positive_tests,cumulative_positive_tests,reported_cases,cumulative_reported_cases,reported_deaths,cumulative_reported_deaths,reported_tests
0,9/5/2022,Alameda,County,1685886.0,0.0,342474.0,0.0,1997.0,NaN,7527816,NaN,421159,859.0,342480.0,1.0,1999.0,16464.0
1,9/5/2022,Alpine,County,1117.0,0.0,133.0,0.0,0.0,NaN,3098,NaN,87,0.0,133.0,0.0,0.0,0.0
2,9/5/2022,Amador,County,38531.0,0.0,9876.0,0.0,89.0,NaN,239692,NaN,11586,22.0,9877.0,0.0,89.0,373.0
3,9/5/2022,Butte,County,217769.0,0.0,38969.0,0.0,453.0,NaN,507312,NaN,43445,118.0,38969.0,0.0,453.0,1283.0
4,9/5/2022,Calaveras,County,44289.0,0.0,8437.0,0.0,133.0,NaN,112496,NaN,10130,15.0,8437.0,0.0,133.0,261.0


In [6]:
# print all possible traits
print("Number of traits: {}".format(len(CaseDeathTest.columns)-1))
print("Including: ")
print(CaseDeathTest.columns)

Number of traits: 16
Including: 
Index(['date', 'area', 'area_type', 'population', 'cases', 'cumulative_cases',
       'deaths', 'cumulative_deaths', 'total_tests', 'cumulative_total_tests',
       'positive_tests', 'cumulative_positive_tests', 'reported_cases',
       'cumulative_reported_cases', 'reported_deaths',
       'cumulative_reported_deaths', 'reported_tests'],
      dtype='object')


In [7]:
# only cares about daily observations: "cases"
CaseDeathTest = CaseDeathTest[["date", "area", "cases"]]

In [8]:
print("Number of missing values before treatment: ", CaseDeathTest.isnull().values.sum())

Number of missing values before treatment:  1010


In [9]:
# counties we are interested in
counties = ["Los Angeles", "San Diego", "San Francisco", "Santa Barbara",\
            "Fresno", "Sacramento", "Ventura", "Riverside"]

In [10]:
# first, make sure numerical traits are recognized
notNum = ["date"]
for trait in CaseDeathTest.columns[CaseDeathTest.columns!="date"]:
    if CaseDeathTest[trait].dtypes != "int64":
        try:
            CaseDeathTest[trait] = pd.to_numeric(CaseDeathTest[trait])
        except Exception:
            notNum.append(trait)
print(notNum)  # keep track of non-numerical traits

['date', 'area']


In [11]:
# create a df of only selected counties, joint by time
df = CaseDeathTest[CaseDeathTest["area"] == counties[0]]
df = df[["date","cases"]]
df = df.rename(columns={"cases" : counties[0]})
for county in counties[1:]:
    # df of only selected counties
    temp = CaseDeathTest[CaseDeathTest["area"] == county]
    # drop "area", rename "cases" with county name
    temp = temp[["date","cases"]]
    temp = temp.rename(columns={"cases" : county})
    df = df.merge(temp, on = "date")

In [12]:
df

,date,Los Angeles,San Diego,San Francisco,Santa Barbara,Fresno,Sacramento,Ventura,Riverside
0,9/5/2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,9/4/2022,44.0,53.0,8.0,21.0,33.0,24.0,26.0,32.0
2,9/3/2022,357.0,199.0,28.0,30.0,81.0,111.0,53.0,134.0
3,9/2/2022,1088.0,337.0,41.0,58.0,149.0,168.0,104.0,288.0
4,9/1/2022,1320.0,446.0,65.0,61.0,161.0,195.0,98.0,329.0
...,...,...,...,...,...,...,...,...,...
944,2/4/2020,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
945,2/3/2020,4.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
946,2/2/2020,4.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
947,2/1/2020,13.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0


In [13]:
print("Number of missing values before treatment: ", df.isnull().values.sum())

Number of missing values before treatment:  1


In [14]:
# helper function: treat missing values
def treat_miss(dataset):
    temp = dataset
    # first, drop value if non-numerical values missing
    for trait in dataset.columns:  # in all traits
        if trait in notNum:
            for i in range(len(dataset[trait])):  # for each observation
                if pd.isnull(dataset.loc[i,trait]):  # if missing data:
                    print("Data Missing after Date: {}".format(dataset["date"].loc[i-1]))
                    temp = temp.drop(labels = i, inplace = False)
    # if missing value is numerical, take average of 2 nearest
    for trait in dataset.columns:  # in all traits
        if trait not in notNum:  # only consider numerical traits
            for i in range(len(temp[trait])):  # for each observation
                if pd.isnull(temp[trait].loc[i]):  # if missing data:
                    preFind, posFind = False, False  # indicator of whether we find nearest value
                    preInd, posInd = 1,1  # indecies we moved
                    try:
                        while not preFind:
                            if not pd.isnull(temp[trait].loc[i-preInd]):  # valid value
                                pre = temp[trait].loc[i-preInd]
                                preFind = True
                            else:
                                preInd += 1
                        while not posFind:
                            if not pd.isnull(temp[trait].loc[i+posInd]):  # valid value
                                pos = temp[trait].loc[i+posInd]
                                posFind = True
                            else:
                                posInd += 1
                    except Exception:  # out of range or so
                        print("warning: check index {}, {}".format(j, i))
                        break
                    temp[trait].loc[i] = np.mean([pre,pos])  # replace null with average of 2 closest
                    print("Data treated on Date: {}".format(dataset["date"].loc[i]))
    return temp

In [15]:
df = treat_miss(df)
df

Data Missing after Date: 2/1/2020


,date,Los Angeles,San Diego,San Francisco,Santa Barbara,Fresno,Sacramento,Ventura,Riverside
0,9/5/2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,9/4/2022,44.0,53.0,8.0,21.0,33.0,24.0,26.0,32.0
2,9/3/2022,357.0,199.0,28.0,30.0,81.0,111.0,53.0,134.0
3,9/2/2022,1088.0,337.0,41.0,58.0,149.0,168.0,104.0,288.0
4,9/1/2022,1320.0,446.0,65.0,61.0,161.0,195.0,98.0,329.0
...,...,...,...,...,...,...,...,...,...
943,2/5/2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
944,2/4/2020,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
945,2/3/2020,4.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
946,2/2/2020,4.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [16]:
# re-order by acending date
df["date"] = pd.to_datetime(df["date"])
df = df.sort_values(by = "date", ignore_index=True)
df

,date,Los Angeles,San Diego,San Francisco,Santa Barbara,Fresno,Sacramento,Ventura,Riverside
0,2020-02-01,13.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0
1,2020-02-02,4.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,2020-02-03,4.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,2020-02-04,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2020-02-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
943,2022-09-01,1320.0,446.0,65.0,61.0,161.0,195.0,98.0,329.0
944,2022-09-02,1088.0,337.0,41.0,58.0,149.0,168.0,104.0,288.0
945,2022-09-03,357.0,199.0,28.0,30.0,81.0,111.0,53.0,134.0
946,2022-09-04,44.0,53.0,8.0,21.0,33.0,24.0,26.0,32.0


In [17]:
# input: original dataset, county we want
# output: all data, sorted by date, of that county
def extract(dataset, county):
    # select data only from the input county
    temp = dataset[["date", county]]
    # reset index
    temp.reset_index(drop=True, inplace=True)
    return temp

In [18]:
Ventura = extract(df, "Ventura")
Ventura

,date,Ventura
0,2020-02-01,0.0
1,2020-02-02,0.0
2,2020-02-03,0.0
3,2020-02-04,0.0
4,2020-02-05,0.0
...,...,...
943,2022-09-01,98.0
944,2022-09-02,104.0
945,2022-09-03,53.0
946,2022-09-04,26.0


In [19]:
# export to csv file and check
df.to_csv("CACases.csv", index = False)  # do not foget to mark version locally